In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

r=requests.get("http://pythonhow.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS")
c=r.content
c

soup=BeautifulSoup(c,"html.parser")
# print(soup.prettify())

all=soup.find_all("div",{"class":"propertyRow"})

all[0].find("h4",{'class':'propPrice'}).text.replace("\n","").replace(" ","")
page_nr=soup.find_all("a",{"class":"Page"})[-1].text
page_nr

In [ ]:
l=[]

base_url="http://pythonhow.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s="
for page in range(0,10*int(page_nr),10):
    r=requests.get(base_url+str(page)+".html")
    c=r.content
    soup=BeautifulSoup(c,"html.parser")
    all=soup.find_all("div",{"class":"propertyRow"})
    

    for item in all:
        d={}

        d["Address"]=item.find_all("span",{'class','propAddressCollapse'})[0].text
        d["Locality"]=item.find_all("span",{'class','propAddressCollapse'})[1].text
        d["Price"]=item.find("h4",{'class','propPrice'}).text.replace("\n","").replace(" ","")

        try:
            d["Beds"]=item.find("span",{"class","infoBed"}).find("b").text
        except:
            d["Beds"]=None

        try:
            d["Area"]=item.find("span",{"class","infoSqFt"}).find("b").text
        except:
            d["Area"]=None

        try:
            d["Full Baths"]=item.find("span",{"class","infoValueFullBath"}).find("b").text
        except:
            d["Full Baths"]=None

        try:
            d["Half Baths"]=item.find("span",{"class","infoValueHalfBath"}).find("b").text
        except:
            pass
        for column_group in item.find_all("div", {"class":"columnGroup"}):
            for feature_group, feature_name in zip(column_group.find_all("span",{"class":"featureGroup"}),
                                                   column_group.find_all("span",{"class":"featureName"})):
                if "Lot Size" in feature_group.text:
                    d["Lot Size"]=feature_name.text
        l.append(d)

In [ ]:
df=pd.DataFrame(l)
df

In [ ]:
df.to_csv("real_estate.csv")